In [2]:
from docscorer.configuration import ScorerConfiguration
from docscorer.docscorer import DocumentScorer
import json
import pandas as pd
config = ScorerConfiguration()
scorer = DocumentScorer(config)
import os

In [13]:
path = "/mnt/d/Prompsit/Datos/HPLTv3_11_2025/hplt-v3-annotated-samples/hbs_Cyrl/hbs_Cyrl.sample.jsonl"
documents = []
with open(path, "r", encoding="utf8") as a:
    for document in a:
        documents.append(json.loads(document))

In [15]:
documents[5]

{'f': './segments/1590347426956.82/warc/CC-MAIN-20200602224517-20200603014517-00013.warc.gz',
 'o': 308316641,
 's': 12731,
 'rs': 119909,
 'u': 'https://desire2.co.uk/sr/technologies',
 'c': 'text/html',
 'ts': '2020-06-02T22:47:10Z',
 'de': 'utf-8',
 'crawl_id': 'CC-MAIN-2020-24',
 'lang': ['srp_Cyrl', 'mkd_Cyrl', 'bjn_Latn'],
 'prob': [0.9998, 0.0002, 0.0],
 'text': "Напредна Пуњење технологија где је УСБ порт скенира уређај (нпр паметни телефон, таблет) и затим подешава се да одговара код пуњења. Овај процес оптимизује капацитет пуњења за сваки Смарт УСБ порта и обезбеђује да се уређај пуни при максималној брзини.\nНапредна безбедност Заштита технологија како би се спречило прегревање, високе струје, и напона-скока у циљу одржавања уређаја сигурно.\nКуалцомм Брзо пуњење ™ 2.0 је свеобухватан пакет технологија за управљање батерије које омогућавају сигурно Куалцомм процесор Снапдрагон ™ уређаји засновани наплатити до 75% брже него уређаји који користе конвенционалне методе пуњења. Б

In [10]:
agrupations = {"ace_arab": "ara_arab", "ary_arab": "ara_arab", "ars_arab":"ara_arab", "acm_arab": "ara_arab", "acm_arab": "ara_arab", "acq_arab":"ara_arab", "aeb_arab":"ara_arab", "apc_arab":"ara_arab", "arb_arab":"ara_arab",
               "ars_arab":"ara_arab", "ary_arab":"ara_arab", "arz_arab":"ara_arab", "arb_arab": "ara_arab",
              "pes_arab": "fas_arab", "prs_arab" : "fas_arab",
              "hrv_latn": "hbs_latn", "bos_latn": "hbs_latn",
               "cmn_hant": "cmn_hans"
              }

path = "/mnt/d/Prompsit/Datos/HPLTv3_11_2025/hplt-v3-annotated-samples"
# for x in os.listdir(path)[-4:]:
for x in ["yue_Hant"]:
    rel_path = os.path.join(path, x)
    save_path = rel_path
    rel_path = os.path.join(rel_path, os.listdir(rel_path)[0])
    documents = []
    with open(rel_path, "r", encoding="utf8") as a:
        for document in a:
            documents.append(json.loads(document))
    
    language = rel_path[-21:-18]
    script = rel_path[-17:-13]
    original_lang_s = f"{language}_{script}"
    # script = "hans" if script == "Hant" else script.lower()
    if f"{language}_{script}" in agrupations:
        new = agrupations[f"{language}_{script}"].split("_")
        language = new[0]
        script = new[1]
    
    df = pd.DataFrame(columns=["id", "results", "text", "language", "old_score", "old_results"])
    for document in documents:
        seg_langs = document["seg_langs"]
        if language in ["ara", "fas", "hbs", "cmn"]:
            seg_langs = [agrupations[x.lower()] if x.lower() in agrupations else x for x in seg_langs]
        
        result = scorer.score_document(ref_lang=language, ref_script=script, lang_segments=seg_langs, document_text=document["text"], doc_id=document["id"], raw_score=False)
        df.loc[df.shape[0]] = [document["id"], result, document["text"], result[2], document["doc_scores"][0]/10, [x/10 for x in document["doc_scores"]]]

    df["WDS"] = df.apply(lambda x: f"{round(x['results'][0], 2)}/{round(x['old_results'][0], 2)}", axis=1)
    df["language"] = df.apply(lambda x: f"{float(round(x['results'][1], 2))}/{float(round(x['old_results'][1], 2))}", axis=1)
    df["urls"] = df.apply(lambda x: f"{float(round(x['results'][2], 2))}/{float(round(x['old_results'][2], 2))}", axis=1)
    df["punct"] = df.apply(lambda x: f"{float(round(x['results'][3], 2))}/{float(round(x['old_results'][3], 2))}", axis=1)
    df["sing_chars"] = df.apply(lambda x: f"{float(round(x['results'][4], 2))}/{float(round(x['old_results'][4], 2))}", axis=1)
    df["numbers"] = df.apply(lambda x: f"{float(round(x['results'][5], 2))}/{float(round(x['old_results'][5], 2))}", axis=1)
    df["repet_seg"] = df.apply(lambda x: f"{float(round(x['results'][6], 2))}/{float(round(x['old_results'][6], 2))}", axis=1)
    df["many_long"] = df.apply(lambda x: f"{float(round(x['results'][7], 2))}/{float(round(x['old_results'][7], 2))}", axis=1)
    df["very_long"] = df.apply(lambda x: f"{float(round(x['results'][8], 2))}/{float(round(x['old_results'][8], 2))}", axis=1)
    df["compression"] = df.apply(lambda x: f"{float(round(x['results'][9], 2))}/{float(round(x['old_results'][9], 2))}", axis=1)
    df["short_seg"] = df.apply(lambda x: f"{float(round(x['results'][10], 2))}/NaN", axis=1)
    df["text"] = df.text.apply(lambda x: x.replace("\n", "\\n").replace("\r", "\\n"))
    df["text"] = df["text"].astype(str)
    df[['id', 'WDS', 'language', 'urls', 'punct', 'sing_chars', 'numbers', 'repet_seg', 'many_long',
        'very_long', 'compression', 'short_seg', 'text'
       ]].to_csv(os.path.join(save_path, f"{original_lang_s}.tsv"), sep="\t",index=False)
        

    print(save_path)

/mnt/d/Prompsit/Datos/HPLTv3_11_2025/hplt-v3-annotated-samples/yue_Hant
